# Rancang Bangun Pengendali Otomatis Lingkungan Termal Berbasis Jaringan Saraf Tiruan

### Program Kecerdasan Buatan untuk membentuk model jaringan saraf tiruan dalam memenuhi Tugas Akhir program studi Teknik Fisika.

In [1]:
# Mengimpor pustaka yang dibutuhkan.

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# Memasukkan dataset ke dalam jupyter.
data = pd.read_excel('Data/dataAll.xlsx')

In [3]:
# # Mengubah tipe data waktu
# def ubahT(dt):
#     waktu = 60*int(dt.strftime('%H')) + int(dt.strftime('%M'))
#     return waktu

# data['Time'] = data['Time'].apply(ubahT)
# data['Time']

In [4]:
# # Mengubah data Month

# def ubahM(dt):
#     if dt == 'Mar':
#         return 3
#     elif dt == 'Jun':
#         return 6
#     elif dt == 'Sep':
#         return 9
#     elif dt == 'Des':
#         return 12

# data['Month'] = data['Month'].apply(ubahM)
# data['Month']

In [5]:
# Memisahkan data input dan data target.

# Data Input
X = data[['Heater','AC','DrybulbT','Radiation']]

# Data Target
target = data[['AirT','RH','No','Variation','Month','Time']]

In [6]:
# Memisahkan data pelatihan (training) dan data pengujian (testing).

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Memisahkan 20% menjadi data tes
X_temp, X_test, y_temp, y_test = train_test_split(X, target, test_size=0.2, shuffle=True, random_state=15)

# Memisahkan 20% menjadi data validation
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, shuffle=True, random_state=2019)

In [7]:
# Menghapus data temporary
del X_temp, y_temp

# print proportions
print('train: {}% | validation: {}% | test {}%'.format(round(len(y_train)/len(target)*100,2),
                                                       round(len(y_val)/len(target)*100,2),
                                                       round(len(y_test)/len(target)*100,2)))

train: 60.0% | validation: 20.0% | test 20.0%


In [8]:
# banyak data
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(14400, 4) (4800, 4) (4800, 4) (14400, 6) (4800, 6) (4800, 6)


## Model Tuning menggunakan Multi-Layer Perceptron Regressor

In [9]:
# Impor pustaka
from sklearn.neural_network import MLPRegressor

# list_neuron = [1,5,10,25,50,75,100]
list_neuron = range(1,21)

In [10]:
# # Mencari optimasi jumlah hidden layer dan neuron terbaik
# import operator

# def ann(m):
#     # Model MLP
#     return MLPRegressor(random_state = 1, activation = 'relu', hidden_layer_sizes=(m), solver='adam', alpha=0.001, max_iter = 5000)

# daftar = {}
# for i in list_neuron:
#     # Pelatihan model
#     model = ann(i)
#     model.fit(X_train, y_train[['AirT','RH']])
    
#     # Prediksi model
#     y_pred = model.predict(X_val)
#     y_pred = pd.DataFrame(y_pred, columns=['AirT','RH'])
    
#     # Menghitung nilai RMSE dan R^2 (Validasi)
#     rmse_AirT = round(mean_squared_error(y_val.AirT, y_pred.AirT)**0.5,2)
#     R2_AirT = round(r2_score(y_val.AirT, y_pred.AirT)*100,2)
    
#     key = str(i)
#     value = str(rmse_AirT) + ',' + str(R2_AirT)
#     daftar[key] = rmse_AirT
#     print(i,': ','RMSE =',rmse_AirT,', R^2 =',R2_AirT,'%')
#     if R2_AirT > 0.5 and rmse_AirT < 1:
#         neuron = {i}
#         break
#     else:
#         neuron = min(daftar.items(), key=operator.itemgetter(1))[0]

# print()
# print(neuron, daftar)  # 15 : RMSE = 1.04, R^2 = 82.61%

In [11]:
lists = sorted(daftar.items())
x, y = zip(*lists)

plt.plot(x, y)
plt.show()

NameError: name 'daftar' is not defined

In [ ]:
# Mencari optimasi jumlah hidden layer dan neuron terbaik
import operator

def ann(m,n):
    # Model MLP
    return MLPRegressor(random_state = 1, activation = 'relu', hidden_layer_sizes=(m,n), solver='adam', alpha=0.001, max_iter = 5000)

daftar = {}
for i in list_neuron:
    for j in list_neuron:
        # Pelatihan model
        model = ann(i,j)
        model.fit(X_train, y_train[['AirT','RH']])

        # Prediksi model
        y_pred = model.predict(X_val)
        y_pred = pd.DataFrame(y_pred, columns=['AirT','RH'])

        # Menghitung nilai RMSE dan R^2 (Validasi)
        rmse_AirT = round(mean_squared_error(y_val.AirT, y_pred.AirT)**0.5,2)
        R2_AirT = round(r2_score(y_val.AirT, y_pred.AirT)*100,2)

        key = str(i) + ',' + str(j)
        value = str(rmse_AirT) + ',' + str(R2_AirT)
        daftar[key] = rmse_AirT
        print(i,j,': ','RMSE =',rmse_AirT,', R^2 =',R2_AirT)
        if R2_AirT > 0.5 and rmse_AirT < 1:
            neuron = {i,j}
            break
        else:
            neuron = min(daftar.items(), key=operator.itemgetter(1))[0]

print()
print(neuron, daftar)  # i, j : RMSE = ...

1 1 :  RMSE = 2.5 , R^2 = -0.01
1 2 :  RMSE = 2.5 , R^2 = -0.01
1 3 :  RMSE = 2.5 , R^2 = -0.01
1 4 :  RMSE = 2.5 , R^2 = -0.01
1 5 :  RMSE = 2.5 , R^2 = -0.02
1 6 :  RMSE = 2.5 , R^2 = -0.0
1 7 :  RMSE = 2.5 , R^2 = -0.03
1 8 :  RMSE = 2.5 , R^2 = -0.0
1 9 :  RMSE = 2.5 , R^2 = -0.0
1 10 :  RMSE = 2.5 , R^2 = -0.0
1 11 :  RMSE = 2.5 , R^2 = -0.03
1 12 :  RMSE = 2.5 , R^2 = -0.0
1 13 :  RMSE = 2.5 , R^2 = -0.0
1 14 :  RMSE = 2.5 , R^2 = -0.01
1 15 :  RMSE = 2.5 , R^2 = -0.0
1 16 :  RMSE = 2.5 , R^2 = -0.07
1 17 :  RMSE = 2.5 , R^2 = -0.01
1 18 :  RMSE = 2.5 , R^2 = -0.01
1 19 :  RMSE = 2.5 , R^2 = -0.03
1 20 :  RMSE = 2.5 , R^2 = -0.0
2 1 :  RMSE = 2.5 , R^2 = -0.01
2 2 :  RMSE = 2.5 , R^2 = -0.01
2 3 :  RMSE = 2.5 , R^2 = -0.01
2 4 :  RMSE = 2.63 , R^2 = -11.04
2 5 :  RMSE = 2.5 , R^2 = -0.01
2 6 :  RMSE = 2.5 , R^2 = -0.05
2 7 :  RMSE = 2.5 , R^2 = -0.05


In [ ]:
# alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
# print('Mencari nilai alpha terbaik')
# print('-'*76)
# for alpha in alphas:
#     # Pembuatan model
#     # model_mlp = MLPRegressor(random_state=1, activation='relu', hidden_layer_sizes=(24,24,24), solver='adam')
#     model_mlp = MLPRegressor(random_state = 1, activation = 'relu', hidden_layer_sizes=(20, 13), solver='adam', 
#                              alpha=alpha, max_iter = 5000)

#     # Melakukan pelatihan (training) menggunakan 'fit'
#     model_mlp.fit(X_train, y_train[['AirT','RH']])

#     # Prediksi model
#     y_pred = model_mlp.predict(X_val)
#     y_pred = pd.DataFrame(y_pred, columns=['AirT','RH'])
    
#     # Menghitung nilai RMSE dan R^2 (Testing)
#     rmse_AirT = round(mean_squared_error(y_val.AirT, y_pred.AirT)**0.5,2)
#     rmse_RH = round(mean_squared_error(y_val.RH, y_pred.RH)**0.5,2)
#     R2_AirT = round(r2_score(y_val.AirT, y_pred.AirT)*100,2)
#     R2_RH = round(r2_score(y_val.RH, y_pred.RH)*100,2)
    

#     # print errors as report
#     print('AirT: RMSE = {}, R^2 = {}%'.format(rmse_AirT,R2_AirT), end=' | ')
#     print('RH: RMSE = {}, R^2 {}% = | alpha: {}'.format(rmse_RH,R2_RH,alpha))

In [ ]:
# Model final
model_mlp = MLPRegressor(random_state = 1, activation = 'relu', hidden_layer_sizes=(15), solver='adam', 
                             alpha=0.001, max_iter = 5000)

# Melakukan pelatihan (training) menggunakan 'fit'
model_mlp.fit(X_train, y_train[['AirT','RH']])

# Prediksi model
y_pred = model_mlp.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=['AirT','RH'])

# Menghitung nilai RMSE dan R^2 (testing)
rmse_AirT = round(mean_squared_error(y_test.AirT, y_pred.AirT)**0.5,2)
rmse_RH = round(mean_squared_error(y_test.RH, y_pred.RH)**0.5,2)
R2_AirT = round(r2_score(y_test.AirT, y_pred.AirT)*100,2)
R2_RH = round(r2_score(y_test.RH, y_pred.RH)*100,2)

print('Testing')
print('AirT : RMSE = {} | R^2 = {}%'.format(rmse_AirT,R2_AirT))
print('RH   : RMSE = {} | R^2 = {}%'.format(rmse_RH,R2_RH))

In [ ]:
print(model_mlp.get_params)

In [ ]:
# Menambahkan fitur No Variation Month dan Time pada hasil prediksi
y_pred.index = y_test.index
y_pred[['No']] = y_test[['No']]
y_pred[['Variation']] = y_test[['Variation']]
y_pred[['Month']] = y_test[['Month']]
y_pred[['Time']] = y_test[['Time']]
y_pred

## Visualisasi

In [ ]:
# Variabel untuk pembuatan grafik

Heater    = X_test['Heater']
AC        = X_test['AC']
DrybulbT  = X_test['DrybulbT']
Radiation = X_test['Radiation']
AirT      = y_test['AirT']
RH        = y_test['RH']

In [ ]:
# plot a line, a perfit predict would all fall on this line
plt.subplot(2, 1, 1)
plt.title('Grafik Heater terhadap AirT dan RH')
plt.plot(Heater, AirT, '.')
plt.plot(Heater, y_pred.AirT, 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(Heater, RH, '.')
plt.plot(Heater, y_pred.RH, 'r.')
plt.xlabel('Heater')
plt.ylabel('RH')

plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Grafik AC terhadap AirT dan RH')
plt.plot(AC, AirT, '.')
plt.plot(AC, y_pred.AirT, 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(AC, RH, '.')
plt.plot(AC, y_pred.RH, 'r.')
plt.xlabel('AC')
plt.ylabel('RH')

plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Grafik DrybulbT terhadap AirT dan RH')
plt.plot(DrybulbT, AirT, '.')
plt.plot(DrybulbT, y_pred.AirT, 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(DrybulbT, RH, '.')
plt.plot(DrybulbT, y_pred.RH, 'r.')
plt.xlabel('DrybulbT')
plt.ylabel('RH')

plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Grafik Radiation terhadap AirT dan RH')
plt.plot(Radiation, AirT, '.')
plt.plot(Radiation, y_pred.AirT, 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(Radiation, RH, '.')
plt.plot(Radiation, y_pred.RH, 'r.')
plt.xlabel('Radiation')
plt.ylabel('RH')

plt.show()

## Visualisation in Time variance

In [ ]:
i = 0
for v in y_test.Variation.unique():
    print('Variasi:', v)
    
    x1  = y_test.loc[y_test['Variation']==v].Time
    
    y1  = X_test.loc[y_test['Variation']==v].AC
    y2  = X_test.loc[y_test['Variation']==v].Heater
    y3  = X_test.loc[y_test['Variation']==v].DrybulbT
    y4  = X_test.loc[y_test['Variation']==v].Radiation
    y5a = y_test.loc[y_test['Variation']==v].AirT
    y5b = y_pred.loc[y_pred['Variation']==v].AirT
    y6a = y_test.loc[y_test['Variation']==v].RH
    y6b = y_pred.loc[y_pred['Variation']==v].RH
    
    plt.subplots(figsize=(30,30))
    
    plt.subplot(3,2,1)
    plt.title('Grafik Nilai-Nilai Variabel terhadap waktu. Variasi: ' + str(v))
    plt.plot(x1, y1, 'o')
    plt.xlabel(None)
    plt.ylabel('AC')
    
    plt.subplot(3,2,2)
    plt.plot(x1, y2, 'o')
    plt.xlabel(None)
    plt.ylabel('Heater')
    
    plt.subplot(3,2,3)
    plt.plot(x1, y3, 'o')
    plt.xlabel(None)
    plt.ylabel('DrybulbT')
    
    plt.subplot(3,2,4)
    plt.plot(x1, y4, 'o')
    plt.xlabel(None)
    plt.ylabel('Radiation')
    
    plt.subplot(3,2,5)
    plt.plot(x1, y5a, 'o', label='target')
    plt.plot(x1, y5b, 'ro', label='prediction')
    plt.xlabel(None)
    plt.ylabel('AirT')
    plt.legend(loc = 'upper left')
    
    plt.subplot(3,2,6)
    plt.plot(x1, y6a, 'o', label='target')
    plt.plot(x1, y6b, 'ro', label='prediction')
    plt.xlabel('waktu')
    plt.ylabel('RH')
    plt.legend(loc = 'upper left')
    
    plt.show()
    
    print('_'*110)
    print()
    
    i+=1
    if i == 3:
        break